# 扑克牌型

## 游戏规则

我们将五张扑克牌的组合称为一手牌，比较一手牌大小来决定胜负。

按照 **先比牌型，再比点数** 的顺序比较牌型的大小

- 牌型大小
   - 同花顺 > 四条 > 葫芦 > 同花 > 顺子 > 三条 > 两对 > 一对 > 散牌
- 点数
    - A > K > Q > J > 10 > 9 > 8 > 7 > 6 > 5 > 4  > 3 > 2  
    但同花顺和顺子中 A 如果配上 2345 时当做 1 点。
    
四条、葫芦、三条、两对、一对先比同样点数张数最多的牌，再比同样点数张数少的牌。例如：两对先比点数较大的对子，再比点数较小的对子，最后才是比单张。<a id="rank_rule"></a>

牌型的分类如下：

|牌型（别名）         | 英文名              | 说明                                    | 范例 |
|:------------------    |: --------------     |: ----------------------------- |: ---- : |
|同花顺                    | Straight Flush   | 五张同一花色且顺连的牌。 |  5♠ 6♠  7♠  8♠  9♠    |
|四条（铁扇）         | Four of a Kind  |  有四张同一点数的牌。        | 4♠  <font color="red">4♥</font>  4♣  <font color="red">4♦</font>  <font color="red">9♥</font>                               |
|葫芦（夫佬）         |  Full house       | 三张同一点数的牌，加一对其他点数的牌。 |  8♠  8♣  <font color="red">8♦</font>  K♠  <font color="red">K♥</font>    |
|同花                        | Flush                 | 五张同一花色的牌。           |   3♠  4♠  8♠  J♠  K♠                             |
|顺子（蛇）             |   Straight           |  五张顺连的牌。                 | A♣  2♣  <font color="red">3♥</font>  <font color="red">4♦</font>  5♠                                |
|三条                        | Three of a kind | 有三张同一点数的牌。        |  7♠  <font color="red">7♥</font>  <font color="red">7♦</font>  2♣  K♦                               |
|两对（滔啤）         |  Two Pairs          | 有两张相同点数的牌，加另外两张相同点数的牌。 |  8♠  <font color="red">8♦</font>  <font color="red">A♥</font>  A♣  J♠         |
|一对（啤）              |   One Pair         | 有两张相同点数的牌。       |  9♠  <font color="red">9♥</font>  4♣  6♠  <font color="red">A♥</font>                               |
|散牌（高牌，乌龙） |   High card      | 不能排成以上组合的牌，以点数决定大小。 | 3♠  <font color="red">6♥</font>  <font color="red">9♦</font>  K♠  A♣        |

## 构造

### 框架

我们要实现的过程 `poker` 是这样的一种过程：
- 从多组手牌中找出最大的一组手牌
- 使用函数 `hand_rank` 对手牌进行分级

> **提示**
>
> max 函数使用关键字 `key` 得出自定义的最值，例如找出列表中绝对值最大的数

In [1]:
max([1, 2, 3, -4, 0], key=abs)

-4

过程 `hand_rank` 将一组牌映射为用数表示的扑克牌型大小，根据 [1.1.2](#rank_rule) 中的规则，写出这一函数。  

In [27]:
def poker(hands):
    """
        从多组牌中返回最好的一手牌
    """
    return max(hands, key=hand_rank)

这里，我们假定
- 得出一组牌点数的 `card_nums` 函数

以及判断一组牌
- 是否为顺子的 `straight` 函数
- 是否同花的 `flush` 函数
- 含有几个相同点数牌的 `kind` 函数
- 是否为两对的 `two_pairs` 函数

已经给出。

In [28]:
def hand_rank_num(hand):
    """ 
        一手扑克牌的牌型大小
    """
    nums = card_nums(hand)
    if straight(nums) and flush(hand): return 8
    elif kind(4, nums): return 7
    elif kind(3, nums) and kind(2, nums): return 6
    elif flush(hand): return 5
    elif straight(nums): return 4
    elif kind(3, nums): return 3
    elif two_pairs(nums): return 2
    elif kind(2, nums): return 1
    else: return 0

上面的函数 `hand_rank_num` 将一手扑克牌分为 9 级 （返回数字 0-8）。  

这种方式无法比较相同牌型的扑克牌的大小，还要根据扑克牌点数的大小继续比较。

--------------------

<center>
<font size="4">7♠	&nbsp;  7♣	 &nbsp; <font color="red">7♦</font> &nbsp;  2♣  &nbsp; <font color="red">K♦</font></font>  
<font size="4">8♠	&nbsp; <font color="red">8♥</font>  &nbsp; <font color="red">4♦</font> &nbsp;  A♠ &nbsp; 8♣</font>
</center>

例如，上面的牌的牌型都是 三条，返回的数字就是相同的。  

我们将上面的牌表示为整数
$$
3071302  \\
3081408
$$
或者是元组
$$ 
(3, 7, 13, 7, 7, 7, 2)\\
(3, 8, 14, 8, 8, 8, 4)
$$

我们将表示扑克牌型的数字放在首位，其后是点数相同的牌，最后表示整组牌的点数，或者是还没有写出的牌的点数。  

表示为元组较好，`max` 将会逐个比较元组中的每个元素，而整数则需要考虑位数的问题，要使不同牌型的表示位数相同。

改写后的 `hand_rank` 函数如下。

In [2]:
def hand_rank(hand):
    """ 
        一手扑克牌的等级，点数
    """
    nums = card_nums(hand)  # 一手牌的点数，由大到小排列
    if straight(nums) and flush(hand):
        return (8, nums[0])  # 同花顺，比较最大的点数
    elif kind(4, nums):
        return (7, kind(4, nums), kind(1, nums))  # 比较  四条 的点数、剩下一张牌的点数
    elif kind(3, nums) and kind(2, nums):
        return (6, kind(3, nums), kind(2, nums))  # 比较  三条 的点数、一对 的点数
    elif flush(hand):
        return (5, nums)  # 同花，比较所有牌的点数
    elif straight(nums):
        return (4, nums[0])  # 顺子，比较最大的点数
    elif kind(3, nums):
        return (3, kind(3, nums), nums)  # 比较  三条 的点数、所有牌的点数
    elif two_pairs(nums):
        return (2, two_pairs(nums), nums)  # 比较 两对 的点数（由大到小），所有牌的点数
    elif kind(2, nums):
        return (1, kind(2, nums), nums)  # 比较 一对 的点数，所有牌的点数
    else:
        return (0, nums)  # 比较所有牌的点数

> **提示**

>这里的 `kind` 函数输入点数、期望点数相同的张数 $n$，输出 $n$ 张点数相同的牌的点数 或 `None`。  
这里将 `kind` 的返回正整数值也用作布尔值。

### 分类

下面，完成 `hand_rank` 中所用到的函数：
- 得出一组牌的点数 `card_nums` 函数
- 是否为顺子的 `straight` 函数
- 是否同花的 `flush` 函数
- 相同点数纸牌的 `kind` 函数
- 两对的 `two_pairs` 函数

`card_nums` 将一组牌的第一位（点数）上的字符映射为对应的数字，并将它们由大到小排列。

In [3]:
def card_nums(hand):
    ch_to_num = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'T': 10, 
                 'J': 11, 'Q': 12, 'K': 13, 'A': 14}
    sort_nums = sorted([ch_to_num[n] for n, s in hand], reverse=True)
    return [5, 4, 3, 2, 1] if (sort_nums == [14, 5, 4, 3, 2]) else sort_nums  # 出现 14(A)5432 时，变为 54321

下面是判断是否为顺子的 straight 函数，以及判断是否同花的 flush 函数。

In [12]:
def straight(nums):
    return nums[0]-nums[-1] == 4 and len(set(nums)) == 5

In [4]:
def flush(hand):
    suits = [s for n, s in hand]  # flush 接收一组牌，不是这一组牌的点数
    return len(set(suits)) == 1

`kind` 函数：输入纸牌的点数、期望点数相同的张数 $n$，输出 $n$ 张点数相同的牌的点数 或 `None`。  

In [6]:
def kind(n, nums):
    for i in nums:
        if nums.count(i) == n:  # 对每个纸牌的点数计数，并找出值等于 n 的点数。
            return i

`two_pairs` 将返回每一对的点数（如果存在），否则返回 `None`。

In [7]:
def two_pairs(nums):
    pair = kind(2, nums)
    if pair:
        i = nums.index(pair) + 1
        sec_pair = kind(2, nums[i:])
        if sec_pair: return pair, sec_pair  # 输入的点数从大到小，输出也是。

### 测试

首先准备好多组纸牌，如同下面列出的那些：
```python
sf = "6C 7C 8C 9C TC".split()  # straight flush
fk = "9D 9H 9S 9C 7D".split()  # four kind
fh = "TC TD TH 7C 7D".split()  # full house
s1 = "AD 2D 3D 4D 5D".split()  # straight A - 5
s2 = "2C 3C 4C 5C 6C".split()  # straight 2 - 6
tp = "KD KH 4C JC JD".split()  # two pairs
tk = "6H 9C AH 9S QD".split()  # one pair
```

需要逐步地测试各个函数
- 输出最大的一组牌 `poker`
    - 不同牌型
    - 仅有一组牌
    - 许多组牌
    
```python
assert poker([sf, fk]) == sf
assert poker([fk, fh, tp]) == fk  # 不同牌型
assert poker([fk]) == fk  # 极端情况：一组牌
assert poker([tp] + [s1] * 99) == s1  # 极端情况：100 组牌
assert poker([s1, s2]) == s2  # 对 A-5 的特殊处理
```

- 测试牌型分级 `hand_rank`

```python
assert hand_rank(sf) == (8, 10)
assert hand_rank(fk) == (7, 9, 7)
assert hand_rank(fh) == (6, 10, 7)
assert hand_rank(s1) == (8, 5)
assert hand_rank(tp) == (2, (13, 11), [13, 13, 11, 11, 4])
assert hand_rank(tk) == (1, 9, [14, 12, 9, 9, 6])
```

- 测试判断一组牌特征 `flush, straight, ...`

```python
assert flush(fk) is False
assert straight(card_nums(fh)) is False
assert kind(4, card_nums(fk)) == 9
assert kind(2, card_nums(fh)) == 7
assert kind(4, card_nums(fh)) is None  # kind 函数的输出情况
assert two_pairs(card_nums(tp)) == (13, 11)  # two_pairs 函数输出两对牌的点数。
assert two_pairs(card_nums(tk)) is None  # two_pairs 函数对于一对牌输出 None
```

### 频率

从一幅牌中得到各个牌型的概率如下表[（链接）](https://zh.wikipedia.org/wiki/%E6%92%B2%E5%85%8B%E7%89%8C%E5%9E%8B)所示。  
而通过大量的实验能够得出牌型的频率。

|同花顺	| 四条	| 葫芦	| 同花	| 顺子	| 三条	| 两对	| 一对	| 散牌 |
|:----  |: ---- |: ---- |: --- |: ----- |: ---  |: ---- |: ---- |: --- |
|0.00154%	| 0.024%	| 0.144%	| 0.197%	| 0.392%	| 2.11%	| 4.75%	| 42.26%	| 50.12% |

首先，写出从一幅牌中随机选择手牌的函数 `take_deal`：
- 将一副按顺序排列的扑克牌打乱顺序
- 从中选出 $n$ 组牌，每组牌默认有五张。

In [10]:
import random

def take_deal(numhands, n=5, deck=[r + s for r in '23456789TJQKA' for s in 'SHDC']):
    """ 从一副牌中，随机地选出 numhands 组牌，每组 n 张牌 """
    random.shuffle(deck)  # shufle 随机交换列表中的元素
    return [deck[n * i:n * i + n] for i in range(numhands)]

下面是利用多次实验计算牌型频率的函数 `hand_rank_freq`。  
实验的次数 $n$ 取值应当保证牌型概率的最小者 同花顺 平均出现 10 次，可取 $n = 700000$。

In [29]:
def hand_rank_freq(n=700*100):
    """ 各个牌型的频率 """
    hand_names = ["散牌", "一对", "两对", "三条", "顺子", "同花", "葫芦", "四条", "同花顺"]
    counts = [0] * 9
    for i in range(n // 10):
        for hand in take_deal(10):
            rank = hand_rank(hand)[0]
            counts[rank] += 1
    for i in reversed(range(9)):
        print("%8s： %7.4f %%" % (hand_names[i], 100. * counts[i] / n))

In [30]:
hand_rank_freq()

     同花顺：  0.0014 %
      四条：  0.0329 %
      葫芦：  0.1471 %
      同花：  0.1971 %
      顺子：  0.4200 %
      三条：  2.0557 %
      两对：  4.7057 %
      一对： 42.0100 %
      散牌： 50.4300 %


## 重构

### allmax

在上面的 poker 函数中，我们用 max 函数选出了一组最大的扑克牌。  
如果要选出所有最大的扑克牌，则可以使用下面的 allmax 函数。

In [1]:
def allmax(lst, key=None):
    key = key or (lambda x: x)  # 如果 key 是默认值 None 则 key 是恒等映射。
    res, maxval = [], None
    for x in lst:
        xval = key(x)
        if not res or xval > maxval:  # 对 maxval = None 的特殊处理
            res, maxval = [x], xval
        elif xval == maxval:
            res.append(x)
    return res

### 分组

我们也可以将扑克牌按照相同点数的张数，点数分组。  
例如，下面的扑克牌分组为

<center><font size="4">7♠	&nbsp;  7♣	 &nbsp; <font color="red">7♦</font> &nbsp;  2♣  &nbsp; <font color="red">K♦</font></font></center>

$$  (3,1,1)\; (7,13,2) $$

下面的 group 函数将一组牌的点数的重复次数分成了多个二元组，例如上面的扑克牌为：

$$ (3, 7), (1, 13), (1, 2) $$

In [2]:
def group(items):
    groups = [(items.count(x), x) for x in set(items)]
    return sorted(groups, reverse=True)

随后，由所分成二元组得出重复次数和对应的点数，并将重复次数映射为牌型的大小。

In [5]:
def hand_rank2(hand):
    groups = group(['--23456789TJQKA'.index(n) for n, s in hand])
    counts, nums = zip(*groups)
    if nums == (14, 5, 4, 3, 2):
        nums = (5, 4, 3, 2, 1)
    is_straight = max(nums)-min(nums)==4 and len(set(nums))==5
    is_flush = len(set([s for n, s in hand])) == 1
    counts_rank = {(5,): 10, (4, 1): 7, (3, 2): 6, (3, 1, 1): 3, (2, 2, 1): 2,
                   (2, 1, 1, 1): 1, (1, 1, 1, 1, 1): 0}
    return max(counts_rank[counts], 4 * is_straight + 5 * is_flush), nums

# 如何洗牌

在 `take_deal` 中，我们使用 `shuffle` 函数打乱纸牌的顺序，来实现洗牌的效果。

## 交换位置

下面的 `shuffle` 函数用于打乱列表中元素的顺序：  
随机地交换未被交换的元素，直到所有元素均被交换为止。

In [66]:
def shuffle1(deck):
    """ 打乱元素的顺序 """
    N = len(deck)
    shuffled = [False] * N
    while not all(shuffled):
        i, j = random.randrange(N), random.randrange(N)
        shuffled[i] = shuffled[j] = True
        swap_it(deck, i, j)

> **警告**

> 函数 `shuffle` 直接修改传入的列表，无返回值。

In [65]:
def swap_it(lst, i, j):
    """ 交换列表中的元素 """
    lst[i], lst[j] = lst[j], lst[i]

这一函数需要多长的时间？  

如果在已经在乱序的列表中添加一个新的元素，如果要使得该元素被交换，则需要的时间为 $n$。  
因此，这个函数所需的时间为 $O(n^2)$

用下面的函数顺序地交换每个位置上的元素，这个函数所需的时间为  $O(n)$。

In [68]:
def shuffle2(lst):
    """ 打乱元素的顺序 """
    N = len(lst)
    for x in range(N - 1):
        swap_it(lst, x, random.randrange(x, N))

## 随机性

上面两个函数有哪些区别？ 它们能否使得每种排列等可能？   
我们进行下面的测试。

In [88]:
from collections import defaultdict

def shuffle_test(shuffler, deck='ABC', n=10000, verbose=True):
    counts = defaultdict(int)
    for _ in range(n):
        input = list(deck)
        shuffler(input)
        counts[''.join(input)] += 1

    def factorial(n):
        return 1 if n < 2 else n * factorial(n - 1)

    mean = n * 1. / factorial(len(deck))
    ok = all(map(lambda x: 0.9 <= x / mean <= 1.1, counts.values()))
    print("%s  \"%s\"  %s" % (shuffler.__name__, deck,
                          ('ok' if ok else '*** bad ***')))
    print("mean: %4.2f" % (mean / 100.), end='    ')
    for item, count in sorted(counts.items()):
        print("%s: %4.2f" % (item, count * 100. / n), end='    ')
    print()

In [91]:
for deck in ('ab', 'abc'):
    print()
    for s in (shuffle1, shuffle2):
        shuffle_test(s, deck)


shuffle1  "ab"  *** bad ***
mean: 50.00    ab: 17.24    ba: 82.76    
shuffle2  "ab"  ok
mean: 50.00    ab: 49.65    ba: 50.35    

shuffle1  "abc"  *** bad ***
mean: 16.67    abc: 4.73    acb: 14.16    bac: 13.94    bca: 26.77    cab: 26.71    cba: 13.69    
shuffle2  "abc"  ok
mean: 16.67    abc: 16.75    acb: 16.66    bac: 16.12    bca: 16.68    cab: 16.91    cba: 16.88    


接着，我们可以写出另一些 `shuffle` 函数，例如下面所展示的片段。
```python
# shuffle3
shuffled[i] = True  # 改写 `shuffle1` 的第 7 行
# shuffle4
for x in range(N):  # 改写 shuffle2的第 4 行
```

由此，画出在不同的 `shuffle` 函数下出现各种情况频率的图像，容易看出其是否具有良好的随机性。

![shuffle1](../assets/shuff1.png)

![shuffle2](../assets/shuff2.png)


![shuffle3](../assets/shuff3.png)

![shuffle4](../assets/shuff4.png)